In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sqlalchemy import create_engine
import sqlite3 as sql
import re
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


%matplotlib inline


In [2]:
police_calls=pd.read_excel("../drive-download-20220404T141044Z-001/Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx")

In [3]:
police_calls

,Precinct,Date,Created,Address,Incident Type,Priority
0,C,07-01-2021,01:57:16,639 LAFAYETTE ST,INTOXICATED PERSON,2
1,C,07-01-2021,03:02:52,920 BROADWAY,WELFARE CHECK,2
2,C,07-01-2021,05:44:47,110 4TH AVE S,DISORDERLY SUBJECT,2
3,C,07-01-2021,05:49:07,800 4TH AVE S,SUSPICIOUS PERSON,2
4,C,07-01-2021,08:01:06,1033 DEMONBREUN ST,TRESPASSER,2
...,...,...,...,...,...,...
48014,W,03-31-2022,11:46:52,3811 HILLSBORO PIKE,TRESPASSER,2
48015,W,03-31-2022,12:24:06,7277 CHARLOTTE PIKE,SUICIDAL PERSON,2
48016,W,03-31-2022,17:59:25,MURPHY RD / 42ND AVE N,DISORDERLY SUBJECT,2
48017,W,03-31-2022,20:31:47,2828 KENWAY RD,SUSPICIOUS PERSON,2


In [4]:
police_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48019 entries, 0 to 48018
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Precinct       48019 non-null  object
 1   Date           47927 non-null  object
 2   Created        47927 non-null  object
 3   Address        47927 non-null  object
 4   Incident Type  47927 non-null  object
 5   Priority       47927 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [5]:
police_calls=police_calls.dropna()

In [6]:
police_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47927 entries, 0 to 48018
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Precinct       47927 non-null  object
 1   Date           47927 non-null  object
 2   Created        47927 non-null  object
 3   Address        47927 non-null  object
 4   Incident Type  47927 non-null  object
 5   Priority       47927 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


In [8]:
#there is an extra row 42490 because when I joined two datasets in excel the header from march dataset got added.
police_calls.loc[police_calls.Precinct=='Precinct']

,Precinct,Date,Created,Address,Incident Type,Priority
42490,Precinct,Date,Created,Address,Incident Type,Priority


In [13]:
#using the index number I can drop the row.

police_calls=police_calls.drop(police_calls.index[42490])

In [18]:
conn = sql.connect("../census.sqlite")
cur = conn.cursor()

In [19]:
# Execute tells the cursor to run the query provided


cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

# fetchall() allows us to see everything returned by our query, which we store in our 'available_tables' variable
available_tables = (cur.fetchall())

In [20]:
print(available_tables)

[('B01001',), ('B03002',), ('B11001',), ('B11002',), ('B15002',), ('B19001',), ('B19013',), ('B19301',), ('B25024',), ('B25075',), ('S1701',)]


In [21]:
# Let's check to see what our data looks like by selecting the first 5 rows

cur.execute("SELECT * FROM B01001 LIMIT 5;")

B01001 = (cur.fetchall())

In [22]:
type(B01001)

list

In [23]:
B01001

[(0,
  '1400000US47037010103',
  'Census Tract 101.03, Davidson County, Tennessee',
  10103,
  37,
  47,
  'B01001_001E',
  2533,
  'Estimate!!Total:'),
 (1,
  '1400000US47037010104',
  'Census Tract 101.04, Davidson County, Tennessee',
  10104,
  37,
  47,
  'B01001_001E',
  2955,
  'Estimate!!Total:'),
 (2,
  '1400000US47037010105',
  'Census Tract 101.05, Davidson County, Tennessee',
  10105,
  37,
  47,
  'B01001_001E',
  4435,
  'Estimate!!Total:'),
 (3,
  '1400000US47037010106',
  'Census Tract 101.06, Davidson County, Tennessee',
  10106,
  37,
  47,
  'B01001_001E',
  3397,
  'Estimate!!Total:'),
 (4,
  '1400000US47037010201',
  'Census Tract 102.01, Davidson County, Tennessee',
  10201,
  37,
  47,
  'B01001_001E',
  4157,
  'Estimate!!Total:')]

In [25]:
# Using the same query as before to select the first 5 rows
pop_by_age_sex_df = pd.read_sql("SELECT * FROM B01001;", conn)

In [26]:
type(pop_by_age_sex_df)

pandas.core.frame.DataFrame

In [62]:
pop_by_age_sex_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B01001_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B01001_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B01001_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B01001_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B01001_001E,4157,Estimate!!Total:


In [30]:
#Now we will use each one of the tables and convert then in datafram and read them in sql also I changed their names.
pop_race_eth_df=pd.read_sql("SELECT* FROM B03002;", conn)

In [31]:
type(pop_race_eth_df)

pandas.core.frame.DataFrame

In [32]:
pop_race_eth_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B03002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B03002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B03002_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B03002_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B03002_001E,4157,Estimate!!Total:


In [34]:
hshld_tp_df=pd.read_sql("SELECT* FROM B11001;", conn)

In [35]:
hshld_tp_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11001_001E,1731,Estimate!!Total:


In [36]:
pop_hshld_tp_df=pd.read_sql("SELECT* FROM B11002;", conn)

In [37]:
pop_hshld_tp_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11002_001E,4430,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11002_001E,3260,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11002_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1927,1927,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B11002_012E,2936,Estimate!!Total:!!In nonfamily households
1928,1928,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B11002_012E,5088,Estimate!!Total:!!In nonfamily households
1929,1929,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B11002_012E,1196,Estimate!!Total:!!In nonfamily households
1930,1930,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B11002_012E,0,Estimate!!Total:!!In nonfamily households


In [41]:
gender_by_ed_for_pop_over_25_df=pd.read_sql("SELECT* FROM B15002;", conn)

In [42]:
gender_by_ed_for_pop_over_25_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B15002_001E,1872,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B15002_001E,2349,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B15002_001E,3621,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B15002_001E,2301,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B15002_001E,3163,Estimate!!Total:


In [45]:
hshld_inc_df=pd.read_sql("SELECT* FROM B19001;", conn)

In [47]:
hshld_inc_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19001_001E,1731,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
2732,2732,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19001_017E,311,"Estimate!!Total:!!$200,000 or more"
2733,2733,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19001_017E,654,"Estimate!!Total:!!$200,000 or more"
2734,2734,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19001_017E,99,"Estimate!!Total:!!$200,000 or more"
2735,2735,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19001_017E,0,"Estimate!!Total:!!$200,000 or more"


In [49]:
median_hshld_inc_df=pd.read_sql("SELECT* FROM B19013;", conn)

In [51]:
median_hshld_inc_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19013_001E,70341,Estimate!!Median household income in the past ...
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19013_001E,64844,Estimate!!Median household income in the past ...
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19013_001E,64688,Estimate!!Median household income in the past ...
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19013_001E,63864,Estimate!!Median household income in the past ...
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19013_001E,62829,Estimate!!Median household income in the past ...


In [52]:
inc_per_cap_df=pd.read_sql("SELECT* FROM B19301;", conn)

In [54]:
inc_per_cap_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19301_001E,33867,Estimate!!Per capita income in the past 12 mon...
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19301_001E,38506,Estimate!!Per capita income in the past 12 mon...
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19301_001E,35122,Estimate!!Per capita income in the past 12 mon...
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19301_001E,30843,Estimate!!Per capita income in the past 12 mon...
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19301_001E,34567,Estimate!!Per capita income in the past 12 mon...


In [56]:
str_typ_df=pd.read_sql("SELECT* FROM B25024;", conn)

In [57]:
str_typ_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25024_001E,1022,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25024_001E,1382,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25024_001E,1958,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25024_001E,1186,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25024_001E,1826,Estimate!!Total:


In [58]:
ownr_occ_val_df=pd.read_sql("SELECT* FROM B25075;", conn)

In [59]:
ownr_occ_val_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25075_001E,873,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25075_001E,975,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25075_001E,1626,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25075_001E,885,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25075_001E,1356,Estimate!!Total:


In [60]:
pov_status_df=pd.read_sql("SELECT* FROM S1701;", conn)

In [61]:
pov_status_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037015631,"Census Tract 156.31, Davidson County, Tennessee",15631,37,47,S1701_C01_001E,12144.0,Estimate!!Total!!Population for whom poverty s...
1,1,1400000US47037015804,"Census Tract 158.04, Davidson County, Tennessee",15804,37,47,S1701_C01_001E,4093.0,Estimate!!Total!!Population for whom poverty s...
2,2,1400000US47037017701,"Census Tract 177.01, Davidson County, Tennessee",17701,37,47,S1701_C01_001E,2466.0,Estimate!!Total!!Population for whom poverty s...
3,3,1400000US47037017702,"Census Tract 177.02, Davidson County, Tennessee",17702,37,47,S1701_C01_001E,3511.0,Estimate!!Total!!Population for whom poverty s...
4,4,1400000US47037018301,"Census Tract 183.01, Davidson County, Tennessee",18301,37,47,S1701_C01_001E,8222.0,Estimate!!Total!!Population for whom poverty s...


In [67]:
# I was trying to find how many mental illness type of incident calls did precinct N got by running a sql query
# but it didnt work because police_calls data is from excel file, sql queries in python will only work if I am trying it 
# using sqlite.
query = '''
SELECT *
FROM police_calls
WHERE Precinct = N AND Incident Type = Mental Illness;
'''

In [68]:
PN=pd.read_sql(query, conn)

DatabaseError: Execution failed on sql '
SELECT *
FROM police_calls
WHERE Precinct = N;
': no such table: police_calls

In [69]:
police_calls.Precinct.value_count()

AttributeError: 'Series' object has no attribute 'value_count'

In [71]:
police_calls['Precinct'].describe()

count     47926
unique       13
top           H
freq       7957
Name: Precinct, dtype: object

In [72]:
police_calls.eval('Precinct==C')[police_calls.isValid].all()

UndefinedVariableError: name 'C' is not defined